## RAG Embeddings Movies Good

In [1]:
#!pip install -q transformers sentence-transformers faiss-cpu 

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.12/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [3]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer
import torch, pandas as pd, faiss, requests, textwrap
from io import StringIO

In [9]:
print("Downloading IMDb movie metadata…")

CSV_URL = (
  "https://raw.githubusercontent.com/"
  "sundeepblue/movie_rating_prediction/master/movie_metadata.csv"
)

try:
  raw = requests.get(CSV_URL, timeout=20)
  raw.raise_for_status()
  df = pd.read_csv(StringIO(raw.text))

# ─── Minimal cleaning ────────────────────────────────────

  df["movie_title"]  = df["movie_title"].str.strip()
  df["genres"]    = df["genres"].str.replace("|", ", ")
  df["plot_keywords"] = df["plot_keywords"].fillna("").str.replace("|", ", ")

  # Unified “text” field for retrieval

  df["text"] = (
    "Title: "    + df["movie_title"] + ". "
    "Director: "   + df["director_name"].fillna("Unknown") + ". "
    "Genres: "    + df["genres"] + ". "
    "Plot keywords: "+ df["plot_keywords"] + "."
  )
  df = df.head(2_000)            # ← keep notebook snappy
  print(f"✅ Loaded {len(df)} movies.")

except Exception as e:
  print(f"❌ Download failed: {e}")
  df = pd.DataFrame()

✅ Loaded 2000 movies.


In [10]:
# ─────────────────────────────────────────────────────────────
# ✅ Cell 4 – Create embeddings & build FAISS inde
# ─────────────────────────────────────────────────────────────

if df.empty:
  raise RuntimeError("Dataset not loaded; aborting.")


print("🔄 Creating embeddings…")
emb_model = SentenceTransformer("all-MiniLM-L6-v2")

embeddings = emb_model.encode(

  df["text"].tolist(),
  batch_size=32,
  show_progress_bar=True
).astype("float32")

index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print(f"✅ FAISS index ready with {index.ntotal} vectors.")

🔄 Creating embeddings…


Batches: 100%|██████████████████████████████████████████████████████████████████████████| 63/63 [00:02<00:00, 23.93it/s]

✅ FAISS index ready with 2000 vectors.


In [11]:
# ─────────────────────────────────────────────────────────────
# ✅ Cell 5 – Load extractive QA model
# ─────────────────────────────────────────────────────────────

print("🧠 Loading QA model…")

qa = pipeline(
  "question-answering",

  model="distilbert-base-cased-distilled-squad",
  device=0 if torch.cuda.is_available() else -1,
)

🧠 Loading QA model…


Device set to use cuda:0


In [12]:
# ─────────────────────────────────────────────────────────────
# ✅ Cell 6 – Helper that returns answer + IDs + distances
# ─────────────────────────────────────────────────────────────

def answer_movie_question(question, k=5, max_ctx_chars=4_500, verbose=True):
  """
  Parameters
  ----------
  question : str
  k    : int  – how many nearest docs to retrieve
  max_ctx_chars : int – truncation length for QA context
  verbose : bool – print pretty output if True 
  Returns
  -------
  answer : str
  ctx   : List[str]  – raw text of the k nearest docs
  ids   : List[int]  – dataframe row-indices
  dists  : List[float] – L2 distances from query vector
  """
  # Embed the question

  q_emb = emb_model.encode([question])[0].astype("float32").reshape(1, -1)

  # Search FAISS

  dists, ids = index.search(q_emb, k)    # ids.shape == (1, k)

  # Build QA context

  ctx = [df["text"][i] for i in ids[0]]
  context = " ".join(ctx)[:max_ctx_chars]

  # Run QA

  try:
    answer = qa(question=question, context=context)["answer"]

  except Exception as e:
    answer = f"Error: {e}"

  # Pretty print (optional)

  if verbose:

    print("\n" + "🔎 Question:".ljust(11), question)
    print("💡 Answer: ".ljust(11), answer, "\n")
    print("📊 Nearest neighbours (rank | row-ID | distance):")
    for r, (i, d) in enumerate(zip(ids[0], dists[0]), 1):
      print(f" {r:>2} | {i:>5} | {d:.4f}")
    print("\n🎞️ Snippets of each context doc:")
    for r, (txt, i, d) in enumerate(zip(ctx, ids[0], dists[0]), 1):
      snip = textwrap.shorten(txt, width=200, placeholder="…")
      print(f"\n--- Doc {r} (row {i} | dist {d:.4f}) ---\n{snip}")

  return answer, ctx, ids[0].tolist(), dists[0].tolist()

In [13]:
# ─────────────────────────────────────────────────────────────
# ✅ Cell 7 – Example query
# ─────────────────────────────────────────────────────────────

_ = answer_movie_question(
  "Who directed Titanic and what genres does it belong to?"
)


🔎 Question: Who directed Titanic and what genres does it belong to?
💡 Answer:   Jerry Jameson 

📊 Nearest neighbours (rank | row-ID | distance):
  1 |    26 | 0.4755
  2 |  1308 | 0.7882
  3 |   606 | 0.9094
  4 |    28 | 0.9206
  5 |  1556 | 0.9220

🎞️ Snippets of each context doc:

--- Doc 1 (row 26 | dist 0.4755) ---
Title: Titanic. Director: James Cameron. Genres: Drama, Romance. Plot keywords: artist, love, ship, titanic, wet.

--- Doc 2 (row 1308 | dist 0.7882) ---
Title: Raise the Titanic. Director: Jerry Jameson. Genres: Action, Adventure, Drama, Thriller. Plot keywords: military, mineral, ocean, ship, titanic.

--- Doc 3 (row 606 | dist 0.9094) ---
Title: The Abyss. Director: James Cameron. Genres: Adventure, Drama, Sci-Fi, Thriller. Plot keywords: alien, estranged couple, ocean, submarine, underwater.

--- Doc 4 (row 28 | dist 0.9206) ---
Title: Battleship. Director: Peter Berg. Genres: Action, Adventure, Sci-Fi, Thriller. Plot keywords: box office flop, hawaii, naval, oahu 

In [ ]:
# ─────────────────────────────────────────────────────────────
# ✅ (Optional) Cell 8 – Interactive loop
# ─────────────────────────────────────────────────────────────
"""
# Uncomment to chat in-notebook
while True:
  q = input("\n🎬 Ask about a movie (or type 'exit'): ")
  if q.lower() in {"exit", "quit"}:
    break
  answer_movie_question(q)

"""